# Загрузка данных анкет и проведение МАИ, анализ полученных результатов
Данные: заполненные анкеты экспертов (таблицы Excel) 

In [1]:
import re
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from random import randint

In [2]:
df = pd.read_excel(r'C:\Users\user\предпосылки\Экс9.xlsx')
df.head()

,Тип нарушения,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала
0,Документация,1.000000,0.200000,0.333333,7,1.000000,0.333333,0.111111
1,Экспертиза промышленной безопасности,5.000000,1.000000,5.000000,9,5.000000,5.000000,5.000000
2,Готовность к ликвидации последствий аварий и и...,3.000000,0.200000,1.000000,5,0.142857,3.000000,0.200000
3,Оформление информационными знаками и знаками б...,0.142857,0.111111,0.200000,1,0.111111,0.333333,0.111111
4,Содержание объектов и оборудования,1.000000,0.200000,7.000000,9,1.000000,7.000000,0.333333


In [3]:
print('Число строк и столбцов в наборе данных:\n', df.shape)

Число строк и столбцов в наборе данных:
 (7, 8)


In [4]:
# типы столбцов 
df.dtypes

Тип нарушения                                                 object
Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности      int64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала       float64
dtype: object

In [5]:
#удаление столбца типа object для дальнейшей работы с фреймом 
df.pop('Тип нарушения') 
print(df)

   Документация   Экспертиза промышленной безопасности  \
0       1.000000                              0.200000   
1       5.000000                              1.000000   
2       3.000000                              0.200000   
3       0.142857                              0.111111   
4       1.000000                              0.200000   
5       3.000000                              0.200000   
6       9.000000                              0.200000   

   Готовность к ликвидации последствий аварий и инцидентов  \
0                                           0.333333         
1                                           5.000000         
2                                           1.000000         
3                                           0.200000         
4                                           7.000000         
5                                           0.333333         
6                                           5.000000         

   Оформление информационными знаками 

In [6]:
# типы столбцов 
df.dtypes

Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности      int64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала       float64
dtype: object

In [7]:
# также меням тип данных Документации
df['Документация '] = df['Документация '].astype(float)

In [8]:
df.loc[:, df.columns[df.dtypes == 'float64']].head(7)

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала
0,1.000000,0.200000,0.333333,1.000000,0.333333,0.111111
1,5.000000,1.000000,5.000000,5.000000,5.000000,5.000000
2,3.000000,0.200000,1.000000,0.142857,3.000000,0.200000
3,0.142857,0.111111,0.200000,0.111111,0.333333,0.111111
4,1.000000,0.200000,7.000000,1.000000,7.000000,0.333333
5,3.000000,0.200000,0.333333,0.142857,1.000000,0.111111
6,9.000000,0.200000,5.000000,3.000000,9.000000,1.000000


In [9]:
DF_1=df[:].copy()

In [10]:
#Найдем сумму всех столбцов, для того чтобы в дальнейшем пересчитать полученные значения
df.sum ()

Документация                                                 22.142857
Экспертиза промышленной безопасности                          2.111111
Готовность к ликвидации последствий аварий и инцидентов      18.866667
Оформление информационными знаками и знаками безопасности    43.000000
Содержание объектов и оборудования                           10.396825
Охранные зоны (ОЗ)                                           25.666667
Организация рабочего процесса и безопасности персонала        6.866667
dtype: float64

In [11]:
# Считаем значения для нашей матрицы, путем деления каждого значения в столбце на сумму столбца
df['new_Документация'] = df['Документация '].astype(float).div(df['Документация '].sum())
df['new_Экспертиза'] = df['Экспертиза промышленной безопасности'].astype(float).div(df['Экспертиза промышленной безопасности'].sum())
df['new_Готовность'] = df['Готовность к ликвидации последствий аварий и инцидентов'].astype(float).div(df['Готовность к ликвидации последствий аварий и инцидентов'].sum())
df['new_Оформление'] = df['Оформление информационными знаками и знаками безопасности'].astype(float).div(df['Оформление информационными знаками и знаками безопасности'].sum())
df['new_Содержание'] = df['Содержание объектов и оборудования '].astype(float).div(df['Содержание объектов и оборудования '].sum())
df['new_ОЗ'] = df['Охранные зоны (ОЗ)'].astype(float).div(df['Охранные зоны (ОЗ)'].sum())
df['new_Организация'] = df['Организация рабочего процесса и безопасности персонала'].astype(float).div(df['Организация рабочего процесса и безопасности персонала'].sum())

In [12]:
df.head()

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,1.000000,0.200000,0.333333,7,1.000000,0.333333,0.111111,0.045161,0.094737,0.017668,0.162791,0.096183,0.012987,0.016181
1,5.000000,1.000000,5.000000,9,5.000000,5.000000,5.000000,0.225806,0.473684,0.265018,0.209302,0.480916,0.194805,0.728155
2,3.000000,0.200000,1.000000,5,0.142857,3.000000,0.200000,0.135484,0.094737,0.053004,0.116279,0.013740,0.116883,0.029126
3,0.142857,0.111111,0.200000,1,0.111111,0.333333,0.111111,0.006452,0.052632,0.010601,0.023256,0.010687,0.012987,0.016181
4,1.000000,0.200000,7.000000,9,1.000000,7.000000,0.333333,0.045161,0.094737,0.371025,0.209302,0.096183,0.272727,0.048544


In [13]:
# Получили новую таблицу теперь отделим значения нашей матрицы от основной таблицы
df_new = df[['new_Документация', 'new_Экспертиза', 'new_Готовность', 'new_Оформление', 'new_Содержание', 'new_ОЗ', 'new_Организация']]
df_new

,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,0.045161,0.094737,0.017668,0.162791,0.096183,0.012987,0.016181
1,0.225806,0.473684,0.265018,0.209302,0.480916,0.194805,0.728155
2,0.135484,0.094737,0.053004,0.116279,0.013740,0.116883,0.029126
3,0.006452,0.052632,0.010601,0.023256,0.010687,0.012987,0.016181
4,0.045161,0.094737,0.371025,0.209302,0.096183,0.272727,0.048544
5,0.135484,0.094737,0.017668,0.069767,0.013740,0.038961,0.016181
6,0.406452,0.094737,0.265018,0.209302,0.288550,0.350649,0.145631


In [14]:
#Вычислим средние значение по строке и запишем полученный результот в mean
df_new['mean'] = df_new.mean(axis=1)
df_new.head()

C:\Users\user\anaconda3\envs\envpy3.5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация,mean
0,0.045161,0.094737,0.017668,0.162791,0.096183,0.012987,0.016181,0.063673
1,0.225806,0.473684,0.265018,0.209302,0.480916,0.194805,0.728155,0.368241
2,0.135484,0.094737,0.053004,0.116279,0.013740,0.116883,0.029126,0.079893
3,0.006452,0.052632,0.010601,0.023256,0.010687,0.012987,0.016181,0.018971
4,0.045161,0.094737,0.371025,0.209302,0.096183,0.272727,0.048544,0.162526


In [15]:
#проверка, среднее значение в сумме должно быть равно 1
df_new['mean'].sum()

1.0

In [16]:
# Далее проведем проверку на согласованность, 
#для этого создадим из нашей полученной выше таблицы матрицы для дальнейшего их перемножения
df_new8 =df_new['mean']

# удаляем из нашей таблицы значениея столбцы mean
#df_new.pop('mean')

df_new1 =DF_1.loc[0]

df_new2 =DF_1.loc[1]

df_new3 =DF_1.loc[2]

df_new4 =DF_1.loc[3]

df_new5 =DF_1.loc[4]

df_new6 =DF_1.loc[5]

df_new7 =DF_1.loc[6]

In [17]:
#Проверка на согласованность
res1 = np.dot(df_new1,df_new8)
print(res1)
res2 = np.dot(df_new2,df_new8)
print(res2)
res3 = np.dot(df_new3,df_new8)
print(res3)
res4 = np.dot(df_new4,df_new8)
print(res4)
res5 = np.dot(df_new5,df_new8)
print(res5)
res6 = np.dot(df_new6,df_new8)
print(res6)
res7 = np.dot(df_new7,df_new8)
print(res7)

0.5056209833805251
3.6029187184496427
0.678585615538606
0.14936801481232576
1.5002002919992723
0.4545888520696195
2.4529365696061562


In [18]:
sum_new = res1 + res2 + res3 + res4 + res5 + res6 + res7
print(sum_new)

9.344219045856146


In [19]:
# Вычислем значения CI, RI, CR
CI = (sum_new - 7)/(7-1)
print(CI)
RI = (1.98*(7-2))/7
print(RI)
CR = CI/RI
print(CR)

0.3907031743093577
1.4142857142857144
0.27625476971368723


In [20]:
# Получилось, что значение CR=0.27, что большее 0,1 значит матрицу нельзя считать согласованной
# Проделаем аналогичные действия с другими экспертами